# Polar Coding Notebook

Helper Functions and Imports

In [19]:
import numpy as np
import matplotlib.pyplot as plt
from math import log2

def is_power_of_two(n):
    return (n & (n-1)) == 0 and n > 0


def bit_reverse_indices(N):
    n = int(np.log2(N))
    br = np.zeros(N, dtype=int)
    for i in range(N):
        br[i] = int(format(i, 'b').zfill(n)[::-1], 2)
    return br


def construct_G_nat(N):
    assert is_power_of_two(N), "N must be power of two"
    F = np.array([[1,0],[1,1]], dtype=int)
    n = int(np.log2(N))
    G = np.array([[1]], dtype=int)
    for _ in range(n):
        G = np.kron(G, F)
    return G % 2

In [20]:
def encode_with_G_nat(u):
    N = len(u)
    G = construct_G_nat(N)
    x = (u.astype(int) @ G.astype(int)) % 2
    return x.astype(np.int8)


def polar_encode_with_G(msg, info_positions, N, u_frozen=None):
    if u_frozen is None:
        u_frozen = np.zeros(N, dtype=np.int8)
    u = np.array(u_frozen, dtype=np.int8)
    u[np.sort(info_positions)] = np.array(msg, dtype=np.int8)
    x = encode_with_G_nat(u)
    return x.astype(np.int8), u.astype(np.int8)


def bhattacharyya_parameter(p, N):
    Z = np.array([2.0 * np.sqrt(p * (1.0 - p))])
    while len(Z) < N:
        Z_next = []
        for z in Z:
            Z_next.append(2*z - z*z)  # Upper
            Z_next.append(z*z) # Lower
        Z = np.array(Z_next)
    return Z


def select_frozen_direct(N, K, p_design):
    Z = bhattacharyya_parameter(p_design, N)
    logical_best = np.argsort(Z)[:K]
    info_positions_natural = np.sort(logical_best)
    frozen_mask = np.ones(N, dtype=bool)
    frozen_mask[info_positions_natural] = False
    return frozen_mask, info_positions_natural


def sc_decode(llr, frozen_mask, u_frozen=None, use_exact_f=False):
    if u_frozen is None:
        u_frozen = np.zeros(len(frozen_mask), dtype=np.int8)
    llr = np.array(llr, dtype=float)
    frozen_mask = np.array(frozen_mask, dtype=bool)
    u_frozen = np.array(u_frozen, dtype=np.int8)

    def rec(llr_sub, frozen_sub, u_frozen_sub):
        n = len(llr_sub)
        if n == 1:
            if frozen_sub[0]:
                return np.array([u_frozen_sub[0]], dtype=np.int8)
            else:
                return np.array([0 if llr_sub[0] >= 0 else 1], dtype=np.int8)
        n2 = n // 2
        a = llr_sub[:n2]; b = llr_sub[n2:]
        f = f_exact(a,b) if use_exact_f else f_min_sum(a,b)
        uhat_up = rec(f, frozen_sub[:n2], u_frozen_sub[:n2])
        sign_term = (1 - 2 * uhat_up)
        g = b + sign_term * a
        uhat_low = rec(g, frozen_sub[n2:], u_frozen_sub[n2:])
        u_upper = uhat_up.astype(np.int8)
        u_lower = (uhat_up ^ uhat_low).astype(np.int8)
        return np.concatenate([u_upper, u_lower]).astype(np.int8)

    return rec(llr, frozen_mask, u_frozen)

Channels

In [21]:
def bsc(x, p):
    flips = (np.random.rand(len(x)) < p).astype(np.int8)
    return (x ^ flips).astype(np.int8)


def bsc_llr(y, p, eps=1e-12):
    p_safe = min(max(float(p), eps), 1.0 - eps)
    alpha = np.log((1.0 - p_safe) / p_safe)
    return (1 - 2*y) * alpha

LLR (Log-Likelihood Ratios) Combines

In [22]:
def f_min_sum(a, b):
    a = np.array(a); b = np.array(b)
    return np.sign(a * b) * np.minimum(np.abs(a), np.abs(b))


def f_exact(a, b, eps=1e-12):
    A = np.array(a, dtype=float)
    B = np.array(b, dtype=float)
    ta = np.tanh(A / 2.0)
    tb = np.tanh(B / 2.0)
    t = ta * tb
    t = np.clip(t, -1.0 + eps, 1.0 - eps)
    return np.log1p(t) - np.log1p(-t)

Tests

In [23]:
def instrumented_n4_matrix(p_design=0.05):
    N = 4; K = 2
    frozen_mask, info_positions = select_frozen_direct(N, K, p_design)
    u_frozen = np.zeros(N, dtype=np.int8)
    msg = np.array([0,1], dtype=np.int8)
    x, u = polar_encode_with_G(msg, info_positions, N, u_frozen=u_frozen)
    print("info_positions (natural):", info_positions)
    print("u (natural):", u.tolist())
    print("x:", x.tolist())
    y = x.copy()
    llr = bsc_llr(y, 1e-12)
    print("llr:", llr.tolist())
    # instrument SC recursion
    def sc_print(llr_sub, frozen_sub, u_frozen_sub, depth=0, path="root"):
        indent = "  "*depth
        n = len(llr_sub)
        print(f"{indent}{path}: n={n}, llr_sub={llr_sub.tolist()}, frozen_sub={frozen_sub.tolist()}")
        if n==1:
            if frozen_sub[0]:
                print(f"{indent} => leaf frozen -> {u_frozen_sub[0]}")
                return np.array([u_frozen_sub[0]], dtype=np.int8)
            dec = 0 if llr_sub[0] >= 0 else 1
            print(f"{indent} => leaf decide -> {dec}")
            return np.array([dec], dtype=np.int8)
        n2 = n//2
        a = llr_sub[:n2]; b = llr_sub[n2:]
        f_llr = f_exact(a,b)
        print(f"{indent} f: a={a.tolist()}, b={b.tolist()} -> f={f_llr.tolist()}")
        uhat_up = sc_print(f_llr, frozen_sub[:n2], u_frozen_sub[:n2], depth+1, path=path+"-L")
        print(f"{indent} uhat_up returned: {uhat_up.tolist()}")
        sign_term = (1 - 2 * uhat_up)
        g_llr = b + sign_term * a
        print(f"{indent} g: sign_term={sign_term.tolist()}, g_llr={g_llr.tolist()}")
        uhat_low = sc_print(g_llr, frozen_sub[n2:], u_frozen_sub[n2:], depth+1, path=path+"-R")
        print(f"{indent} uhat_low returned: {uhat_low.tolist()}")
        combined = np.concatenate([uhat_up.astype(np.int8), (uhat_up ^ uhat_low).astype(np.int8)]).astype(np.int8)
        print(f"{indent} combined: {combined.tolist()}")
        return combined
    uhat = sc_print(llr, frozen_mask, u_frozen)
    print("decoded uhat:", uhat.tolist())
    print("expected u:", u.tolist())
    print("match?", np.array_equal(uhat, u))
    return uhat, u, frozen_mask, info_positions


In [24]:
uhat, u_expected, frozen_mask_ex, info_pos = instrumented_n4_matrix()

info_positions (natural): [2 3]
u (natural): [0, 0, 0, 1]
x: [1, 1, 1, 1]
llr: [-27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111592755]
root: n=4, llr_sub=[-27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111592755], frozen_sub=[True, True, False, False]
 f: a=[-27.63102111592755, -27.63102111592755], b=[-27.63102111592755, -27.63102111592755] -> f=[26.937840546492907, 26.937840546492907]
  root-L: n=2, llr_sub=[26.937840546492907, 26.937840546492907], frozen_sub=[True, True]
   f: a=[26.937840546492907], b=[26.937840546492907] -> f=[26.244693365930964]
    root-L-L: n=1, llr_sub=[26.244693365930964], frozen_sub=[True]
     => leaf frozen -> 0
   uhat_up returned: [0]
   g: sign_term=[1], g_llr=[53.875681092985815]
    root-L-R: n=1, llr_sub=[53.875681092985815], frozen_sub=[True]
     => leaf frozen -> 0
   uhat_low returned: [0]
   combined: [0, 0]
 uhat_up returned: [0, 0]
 g: sign_term=[1, 1], g_llr=[-55.2620422318551, -55.2620422318551]
